Reference:  
https://github.com/ACloudGuru-Resources/Course_AWS_Certified_Machine_Learning/blob/master/Chapter6/ufo-modeling-lab.ipynb


## UFO Sightings K-Means Clustering
### Modeling Lab

The goal of this notebook is to analyze where Mr. K should build his extraterrestrial life facilities using the K-Means algorithm. 

What we plan on accomplishing is the following:
1. [Load dataset onto Notebook instance from S3](#Step-1:-Loading-the-data-from-Amazon-S3)
2. [Cleaning, transforming, and preparing the data](#Step-2:-Cleaning,-transforming,-and-preparing-the-data)
3. [Create and train our model](#Step-3:-Create-and-train-our-model)
4. [Viewing the results](#Step-4:-Viewing-the-results)
5. [Visualize using QuickSight](https://docs.aws.amazon.com/quicksight/latest/user/create-a-data-set-s3.html)

In [ ]:
import pandas as pd
import numpy as np
from datetime import datetime

import boto3
from sagemaker import get_execution_role
import sagemaker.amazon.common as smac

First let's go ahead and import all the needed libraries.

## Step 1: Loading the data from Amazon S3
Next, lets get the UFO sightings data that is stored in S3.

In [18]:
# role = get_execution_role()

def resolve_sm_role():
    client = boto3.client('iam', region_name='us-east-2')
    response_roles = client.list_roles(
        PathPrefix='/',
        # Marker='string',
        MaxItems=999
    )
    for role in response_roles['Roles']:
        if role['RoleName'].startswith('AmazonSageMaker-ExecutionRole-'):
#             print('Resolved SageMaker IAM Role to: ' + str(role))
            return role['Arn']
    raise Exception('Could not resolve what should be the SageMaker role to be used')

# this is the role created by sagemaker notebook on aws
role_arn = resolve_sm_role()
print(role_arn)

role = role_arn



arn:aws:iam::868024899531:role/service-role/AmazonSageMaker-ExecutionRole-20200223T140135


In [7]:
import io
# import s3fs
bucket = 'md-labs-ml-bucket'
prefix = 'acg'
data_key = 'ufo_fullset.csv'
data_location = '{}/{}'.format(prefix, data_key)


s3_client = boto3.client('s3')
response = s3_client.get_object(Bucket=bucket, Key=data_location)
response_body = response['Body'].read()
df = pd.read_csv(io.BytesIO(response_body), header=0, delimiter=',', low_memory=False)
df.head()

,reportedTimestamp,eventDate,eventTime,shape,duration,witnesses,weather,firstName,lastName,latitude,longitude,sighting,physicalEvidence,contact,researchOutcome
0,1977-04-04T04:02:23.340Z,1977-03-31,23:46,circle,4,1,rain,Ila,Bashirian,47.329444,-122.578889,Y,N,N,explained
1,1982-11-22T02:06:32.019Z,1982-11-15,22:04,disk,4,1,partly cloudy,Eriberto,Runolfsson,52.664913,-1.034894,Y,Y,N,explained
2,1992-12-07T19:06:52.482Z,1992-12-07,19:01,circle,49,1,clear,Miller,Watsica,38.951667,-92.333889,Y,N,N,explained
3,2011-02-24T21:06:34.898Z,2011-02-21,20:56,disk,13,1,partly cloudy,Clifton,Bechtelar,41.496944,-71.367778,Y,N,N,explained
4,1991-03-09T16:18:45.501Z,1991-03-09,11:42,circle,17,1,mostly cloudy,Jayda,Ebert,47.606389,-122.330833,Y,N,N,explained


In [8]:
df.shape

(18000, 15)

## Step 2: Cleaning, transforming, and preparing the data
Create another DataFrame with just the latitude and longitude attributes

In [9]:
df_geo = df[['latitude', 'longitude']]

In [10]:
df_geo.head()

,latitude,longitude
0,47.329444,-122.578889
1,52.664913,-1.034894
2,38.951667,-92.333889
3,41.496944,-71.367778
4,47.606389,-122.330833


In [11]:
df_geo.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 18000 entries, 0 to 17999
Data columns (total 2 columns):
latitude     18000 non-null float64
longitude    18000 non-null float64
dtypes: float64(2)
memory usage: 281.3 KB


In [12]:
missing_values = df_geo.isnull().values.any()
print('Are there any missing values? {}'.format(missing_values))

Are there any missing values? False


In [13]:
if(missing_values):
    df_geo[df_geo.isnull().any(axis=1)]

transform the pandas DataFrame (our dataset) into a numpy.ndarray. When we do this each row is converted to a Record object. According to the documentation, this is what the K-Means algorithm expects as training data. This is what we will use as training data for our model.

[See the documentation for input training](https://sagemaker.readthedocs.io/en/stable/kmeans.html)

In [14]:
data_train = df_geo.values.astype('float32')
data_train

array([[  47.329445, -122.57889 ],
       [  52.664913,   -1.034894],
       [  38.951668,  -92.333885],
       ...,
       [  36.86639 ,  -83.888885],
       [  35.385834,  -94.39833 ],
       [  29.883055,  -97.94111 ]], dtype=float32)

## Step 3: Create and train our model
In this step we will import and use the built-in SageMaker K-Means algorithm. We will set the number of cluster to 10 (for our 10 sensors), specify the instance type we want to train on, and the location of where we want our model artifact to live. 

[See the documentation of hyperparameters here](https://docs.aws.amazon.com/sagemaker/latest/dg/k-means-api-config.html)

In [15]:
from sagemaker import KMeans

num_clusters = 10
output_location = 's3://' + bucket + '/model-artifacts'

kmeans = KMeans(
    role=role,
    train_instance_count=1,
    train_instance_type='ml.c4.xlarge',
    output_path=output_location,
    k=num_clusters)

In [20]:
job_name = 'kmeans-geo-job-{}'.format(datetime.now().strftime("%Y%m%d%H%M%S"))
print('Here is the job name {}'.format(job_name))

Here is the job name kmeans-geo-job-20200225202354


In [21]:
%%time
kmeans.fit(kmeans.record_set(data_train), job_name=job_name)

2020-02-25 20:24:04 Starting - Starting the training job...
2020-02-25 20:24:05 Starting - Launching requested ML instances...
2020-02-25 20:25:00 Starting - Preparing the instances for training......
2020-02-25 20:26:04 Downloading - Downloading input data...
2020-02-25 20:26:30 Training - Downloading the training image..Docker entrypoint called with argument(s): train
[02/25/2020 20:26:46 INFO 139704223979328] Reading default configuration from /opt/amazon/lib/python2.7/site-packages/algorithm/resources/default-input.json: {u'_enable_profiler': u'false', u'_tuning_objective_metric': u'', u'_num_gpus': u'auto', u'local_lloyd_num_trials': u'auto', u'_log_level': u'info', u'_kvstore': u'auto', u'local_lloyd_init_method': u'kmeans++', u'force_dense': u'true', u'epochs': u'1', u'init_method': u'random', u'local_lloyd_tol': u'0.0001', u'local_lloyd_max_iter': u'300', u'_disable_wait_to_read': u'false', u'extra_center_factor': u'auto', u'eval_metrics': u'["msd"]', u'_num_kv_servers': u'1', 

## Step 4: Viewing the results
In this step we will take a look at the model artifact SageMaker created for us and stored onto S3. We have to do a few special things to see the latitude and longitude for our 10 clusters (and the center points of those clusters)

[See the documentation of deserilization here](https://docs.aws.amazon.com/sagemaker/latest/dg/cdf-training.html#td-deserialization)

At this point we need to "deserilize" the model artifact. Here we are going to open and review them in our notebook instance. We can unzip the model artifact which will contain model_algo-1. This is just a serialized Apache MXNet object. From here we can load that serialized object into a numpy.ndarray and then extract the clustered centroids from the numpy.ndarray.

After we extract the results into a DataFrame of latitudes and longitudes, we can create a CSV with that data, load it onto S3 and then visualize it with QuickSight.

In [22]:
import os
model_key = 'model-artifacts/' + job_name + '/output/model.tar.gz'

boto3.resource('s3').Bucket(bucket).download_file(model_key, 'model.tar.gz')
os.system('tar -zxvf model.tar.gz')
os.system('unzip model_algo-1')

2304

In [24]:
import sys
! echo {sys.prefix}
! {sys.prefix}/bin/pip install mxnet

/home/ec2-user/anaconda3/envs/python3


In [25]:
import mxnet as mx
Kmeans_model_params = mx.ndarray.load('model_algo-1')

In [26]:
Kmeans_model_params

[
 [[  35.42542     -98.16521   ]
  [  51.676582     -0.17054749]
  [  38.868782    -85.779526  ]
  [ -11.016346    126.31821   ]
  [  47.873764   -122.32976   ]
  [  41.03354     -75.01864   ]
  [  27.944923     45.62018   ]
  [  35.678303   -117.56783   ]
  [  21.241566   -157.69286   ]
  [  25.837196    -80.32623   ]]
 <NDArray 10x2 @cpu(0)>]

In [27]:
cluster_centroids_kmeans = pd.DataFrame(Kmeans_model_params[0].asnumpy())
cluster_centroids_kmeans.columns=df_geo.columns
cluster_centroids_kmeans

,latitude,longitude
0,35.425419,-98.165207
1,51.676582,-0.170547
2,38.868782,-85.779526
3,-11.016346,126.318207
4,47.873764,-122.329758
5,41.033539,-75.018639
6,27.944923,45.620178
7,35.678303,-117.567833
8,21.241566,-157.692856
9,25.837196,-80.326233


Let's go ahead and upload this dataset onto S3 and view within QuickSight

In [28]:
from io import StringIO

csv_buffer = StringIO()
cluster_centroids_kmeans.to_csv(csv_buffer, index=False)
s3_resource = boto3.resource('s3')
s3_resource.Object(bucket, 'results/ten_locations_kmeans.csv').put(Body=csv_buffer.getvalue())

{'ResponseMetadata': {'RequestId': '8ACAC031D95FA14A',
  'HostId': '79ftHn2DE/Dxa4AvxEovRdvFxdw8S97FhsdcO8TQUuZzHXbzQLw6jmC+xQX5MDpkH/IogzKAN88=',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amz-id-2': '79ftHn2DE/Dxa4AvxEovRdvFxdw8S97FhsdcO8TQUuZzHXbzQLw6jmC+xQX5MDpkH/IogzKAN88=',
   'x-amz-request-id': '8ACAC031D95FA14A',
   'date': 'Tue, 25 Feb 2020 20:51:05 GMT',
   'etag': '"638f3c45878318dd9958420040dbdc42"',
   'content-length': '0',
   'server': 'AmazonS3'},
  'RetryAttempts': 0},
 'ETag': '"638f3c45878318dd9958420040dbdc42"'}